# Boarding Passengers

Very interesting problem about probability

**Question** : There are 64 people waiting to board an airplane, each with an assigned seat. The first to board has lost his boarding pass, and sits in a random seat. Every subsequent passenger has their boarding pass. If they find their assigned seat empty, they sit in it.  Otherwise, they randomly pick an open seat.  What is the probability the last passenger to board sits in his/her assigned seat?

Before trying to rigourously answer the question, let's run some Monte Carlo analysis and simulate the boarding process described. This will give us an idea of the probability.

### Monte Carlo simulation

The function below implements the boarding process described in the question. The first person to board chooses a seat at random. Then, one by one the rest of the passengers try to get their allocated seat. If it is still empty, they will sit; if it has been taken, they choose another one at random.

In the end, we check whether the last person to board can actually sit in the correct seat. We run this random process many times, and calculate the probability in a frequentist way: the number of successful trials over the total number.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def arrange_seats(N_seats):
    """
    Simulate a run of boarding passengers
    where the first passengers forget his/her boarding pass
    and chooses a seat at random.
    
    The remaining passengers will choose their seat (if available),
    if not they will choose a random empty seat.
    
    We want to calculate the probability that the last person
    to board gets their assigned seat.
    """
    
    # Number people 
    people = np.arange(0, N_seats)

    # randomly shuffle them
    np.random.shuffle(people)
    available_seats = list(people)
    first = people[0]
    first_chooses = np.random.choice(available_seats, size=1)[0]
#     print("Person #%d chooses Seat #%d" % (first, first_chooses))
    
    # Remove that choice from the list of available seats
    available_seats.remove(first_chooses)
    people_left = list(people).copy()
    people_left.remove(first)
#     print("Available seats: ", available_seats)
#     print("People left: ", people_left)
    
    # Board the remaining passengers except the last
    for person in people[1:-1]:
        
        # Their seat is empty. Occupy it
        if person in available_seats:
#             print("\nPerson #%d chooses Seat #%d" % (person, person))
            available_seats.remove(person)
            people_left.remove(person)
#             print("Available seats: ", available_seats)
#             print("People left: ", people_left)

        # Their seat is not empty. Choose a random one
        else:
            chooses = np.random.choice(available_seats, size=1)[0]
#             print("\nSeat #%d not available, Person #%d chooses Seat #%d" % (person, person, chooses))
            available_seats.remove(chooses)
            people_left.remove(person)
#             print("Available seats: ", available_seats)
#             print("People left: ", people_left)

    # Finally check if the last person's seat is free
    if people_left[0] == available_seats[0]:
        return 1
    else:
        return 0

Let's run this many times

In [4]:
N_seats = 64
N_trials = 5000
total = 0
for k in range(N_trials):
    total += arrange_seats(N_seats)
    
pcent = 100 * total / N_trials

print("%d / %d [%.2f per cent]" % (total, N_trials, pcent))

2546 / 5000 [50.92 per cent]


## Mathematical proof

The Monte Carlo simulation seems to suggest that the probability is exactly 50%. To me, this was a very surprising result. Initially, I thought that the chances of the last person finding his/her seat available would be relatively low; definitely not a 50-50 chance. But when you actually work through the mathematics of it, the result makes a lot of sense.


### Toy problem
To illustrate the reasoning behind it, let's begin with a smaller version of the problem: a plane with 3 seats. Three people (**A**, **B**, and **C**) have their seats allocated as **1**, **2**, and **3**. We can find the probability by working through all the possible outcomes:

The first person **A** forgets that he/she should seat in **1**, and chooses a seat at random.
- If **A** chooses **1** (their assigned seat), then we have no problem and the last person gets to sit in the correct seat. Probability 1/3
- If **A** chooses **2**, then **B** will find that their seat is no longer empty and will choose one seat at random, out of the available ones: **1** and **3**. Probability 1/3
    - If **B** chooses **1**, then it works out because **C** can seat in **3**. Probability 1/2
    - If **B** chooses **3**, then it fails because **C** cannot seat in the assigned seat. Probability 1/2
- If **A** chooses **3**, it also fails because **C** won't find that seat empty. Probability 1/3

This can be written as:
\begin{equation}
P(C, 3) = P(A, 1) + P(A, 2) \cdot P(B, 1) = \frac{1}{3} + \frac{1}{3}\cdot\frac{1}{2} = \frac{3}{6}
\end{equation}

We can see the pattern here. The probability of the last person getting his/her seat can be split into two terms: **the probability of the first person randomly choosing the correct seat** plus the **probability of the first person randomly choosing the incorrect seat** times the **probability of the rest of the passengers occupying the first person's seat**.

In a sense, there are only two positive outcomes: either the first person actually chooses the correct seat or they choose the incorrect seat, but someone else "fixes that mistake" by occupying the seat that was assigned to the first person.

### General case
The first probability (**first person randomly choosing the correct seat**) is rather simple; it is just $1/N$ with $N$ being the number of seats / people.

For the second term, the **probability of the first person randomly choosing the incorrect seat** is given by $(N - 2) / N$ since there are $N$ seats available, but he/she has to choose out of all seats except his/her seat and the last person's seat.

The difficult bit here is how to calculate the **probability of the rest of the passengers occupying the first person's seat**. Honestly, this took me some time to crack... This probability seems very difficult to calculate because whether an arbitrary passenger gets to sit in the first person's seat depends on the number of seats available for that passenger, which depends on the position of that person in the boarding queue and on whether that seat was already taking by previous passengers.

But if you think about it from a different perspective, it becomes much simpler. In this situation, where the first passenger chose the incorrect seat thus disturbing the normal boarding, the only thing that matters is whether someone occupies the first person's seat or the last person's seat. 

If someone occupies the first person's seat then it is guaranteed that the last person will get the correct seat. In contrast, if someone takes the last person's seat, then it is guaranteed to fail. And since both seats are equally likely to be occupied (since people choose at random), the **probability of the rest of the passengers occupying the first person's seat** is actually 50%. In half the cases, someone will "fix the mistake" by taking the first person's seat and the boarding will carry on smoothly. In the other half, someone will take the last person's seat and again the boarding will carry on smoothly but the last person will find that seat occupied.

Putting everything together, the probability of the last person $N$ sitting in their seat $s_N$ can be written as:

\begin{equation}
P(N, s_N) = P(1, s_1) + P(1, s_k) \cdot P(k, s_1) = \frac{1}{N} + \frac{N-2}{N} \cdot \frac{1}{2} = \frac{2 + N - 2}{2 N} = \frac{1}{2}
\end{equation}

Which is always 50%, independent of the number of people boarding the plane.


